In [1]:
%matplotlib inline
import matplotlib
from matplotlib import pyplot 
import numpy #as np
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16


# Постановка задачи

Задача Римана для одномерной нестационарной системы уравнений Эйлера прелставлеяет собой задачу Коши для законов сохранения

\begin{equation}
\left . 
\begin{array}{c}
\mathbf{U}_t + \mathbf{F}(\mathbf{U})_x = 0, \\
\mathbf{U} = \left[\begin{array}{c}\rho \\ \rho u  \\E \end{array}\right], 
\mathbf{F} = \left[\begin{array}{c}\rho u \\ \rho u^2+p  \\ u(E+p) \end{array}\right],
\end{array}
\right\}
\end{equation}

с начальными условиями

\begin{equation}
\mathbf{U}(x,0)=\mathbf{U}^{(0)}(x) = 
\left \{ 
\begin{array}{c}
\mathbf{U}_L ~\textrm{if} ~x<0, \\
\mathbf{U}_R ~\textrm{if} ~x>0.
\end{array}
\right .
\end{equation}

При решении задачи Римана будем использовать вектор примитивных переменных $\mathbf{W} = (\rho, u, p)^T$ вместо консервативных переменных (вектор $\mathbf{U}$)

In [2]:
from IPython.core.display import HTML
css_file = '../../styles/numericalmoocstyle.css'
HTML(open(css_file, "r").read())